<!-- ![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/refs/heads/main/docs/assets/images/logo.png) -->
![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/end-to-end-notebooks/JohnSnowLabs_RealWorld_Notebook.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest on John Snow Labs

In [ ]:
!pip install langtest[johnsnowlabs]

# Harness and its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [3]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>



| Parameter  | Description |  
| - | - | 
|**task**     |Task for which the model is to be evaluated (text-classification or ner)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |


<br/>
<br/>

# Real-World Project Workflows

In this section, we dive into complete workflows for using the model testing module in real-world project settings.

## Robustness Testing

In this example, we will be testing a model's robustness. We will be applying 2 tests: add_typo and lowercase. The real-world project workflow of the model robustness testing and fixing in this case goes as follows:

1. Train NER model on original CoNLL training set

2. Test NER model robustness on CoNLL test set

3. Augment CoNLL training set based on test results 

4. Train new NER model on augmented CoNLL training set

5. Test new NER model robustness on the CoNLL test set from step 2

6. Compare robustness of new NER model against original NER model

#### Load Train and Test CoNLL

In [ ]:
# Load test CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

# Load train CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll

#### Step 1: Train NER Model

In [5]:
from johnsnowlabs import nlp

In [6]:
ner_model = nlp.load('bert train.ner').fit(dataset_path="/content/conll03.conll")


small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


#### Step 2: Test NER Model Robustness  

In [17]:
harness = Harness(task="ner", model={"model": ner_model, "hub": "johnsnowlabs"}, data={"data_source" :"sample.conll"})

In [18]:
harness.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.65},
        
        'robustness': {
            'add_typo': {'min_pass_rate': 0.65}, 
            'lowercase':{'min_pass_rate': 0.65},
        }
    }
})

{'tests': {'defaults': {'min_pass_rate': 0.65},
 'robustness': {'add_typo': {'min_pass_rate': 0.65},
   'lowercase': {'min_pass_rate': 0.65}}}}

Here we have configured the harness to perform two robustness tests (add_typo and lowercase) and defined the minimum pass rate for each test.


#### Generating the test cases.




In [19]:
harness.generate()

harness.generate() method automatically generates the test cases (based on the provided configuration)

In [20]:
harness.testcases()

,category,test_type,original,test_case,expected_result
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , DHINA IN SURPRI...","JAPAN: B-LOC, CHINA: B-PER"
1,robustness,add_typo,Nadim Ladki,Padim Ladki,"Nadim: B-PER, Ladki: I-PER"
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , United Arab Emirates1 996-12-06","AL-AIN: B-LOC, United: B-LOC, Arab: I-LOC, Emi..."
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the sefence of their Asian Cup tit...,"Japan: B-LOC, Asian: B-MISC, Cup: I-MISC, Syri..."
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"China: B-LOC, Uzbekistan: B-LOC"
...,...,...,...,...,...
447,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0,"Portuguesa: B-ORG, Atletico: B-ORG, Mineiro: I..."
448,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .,LARA: B-PER
449,robustness,lowercase,Robert Galvin,robert galvin,"Robert: B-PER, Galvin: I-PER"
450,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06,MELBOURNE: B-LOC


harness.testcases() method gives the produced test cases in form of a pandas data frame.

#### Saving test configurations, data, test cases

In [21]:
harness.save("saved_test_configurations")

#### Running the tests

In [22]:
harness.run()

Running test cases...: 100%|██████████| 452/452 [00:58<00:00,  7.66it/s]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

In [23]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , DHINA IN SURPRI...","japan: LOC, china: LOC","japan: LOC, dhina: PER",False
1,robustness,add_typo,Nadim Ladki,Padim Ladki,nadim ladki: PER,padim ladki: PER,True
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , United Arab Emirates1 996-12-06","al-ain: LOC, united arab emirates: LOC","al-ain: LOC, united arab emirates1: LOC",False
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the sefence of their Asian Cup tit...,"japan: LOC, asian cup: MISC, syria: LOC","japan: LOC, asian cup: MISC, syria: LOC",True
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"china: LOC, uzbekistan: LOC","china: LOC, matsh: PER, uzbekistan: LOC",False
...,...,...,...,...,...,...,...
447,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0,"portuguesa: ORG, atletico mineiro: ORG","portuguesa: ORG, atletico mineiro: ORG",True
448,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .,lara endures: PER,lara endures: PER,True
449,robustness,lowercase,Robert Galvin,robert galvin,robert galvin: PER,robert galvin: PER,True
450,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06,melbourne: LOC,melbourne: LOC,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

#### Report of the tests

In [24]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,98,128,57%,65%,False
1,robustness,lowercase,0,226,100%,65%,True


It summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

#### Step 3: Augment CoNLL Training Set Based on Robustness Test Results

In [25]:
data_kwargs = {
      "data_source" : "conll03.conll",
       }

harness.augment(training_data=data_kwargs, save_data_path="augmented.conll", export_mode="add")

Essentially it applies perturbations to the input data based on the recommendations from the harness reports. Then this augmented_dataset is used to retrain the original model so as to make the model more robust and improve its performance.

#### Step 4: Train New NER Model on Augmented CoNLL

In [26]:
augmented_ner_model = nlp.load('bert train.ner').fit(dataset_path= "augmented.conll")

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


#### Load saved test configurations, data

In [27]:
harness = harness = Harness.load("saved_test_configurations",
                                 model={"model":augmented_ner_model,"hub":"johnsnowlabs"},
                                 task = "ner",
                                 load_testcases=True)

Generating testcases... (robustness): 100%|██████████| 1/1 [00:30<00:00, 30.37s/it]


#### Step 5: Test New NER Model Robustness

In [28]:
harness.run()

Running test cases...: 100%|██████████| 452/452 [00:59<00:00,  7.55it/s]


In [29]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , CHINA IN SYRPRI...","japan: LOC, china: LOC","japan: LOC, china: LOC",True
1,robustness,add_typo,Nadim Ladki,Nasim Ladki,nadim ladki: PER,nasim ladki: PER,True
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , United Arsb Emirates 1996-12-06","al-ain: LOC, united: LOC, arab emirates: LOC","al-ain: LOC, united arsb emirates: LOC",False
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...,"japan: LOC, asian cup: MISC, syria: LOC","japan: LOC, asian cup: MISC, syria: LOC",True
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck dseert them in the se...,"china: LOC, uzbekistan: LOC","china: LOC, uzbekistan: LOC",True
...,...,...,...,...,...,...,...
447,robustness,lowercase,Portuguesa 1 Atletico Mineiro 0,portuguesa 1 atletico mineiro 0,"portuguesa: ORG, atletico mineiro: ORG","portuguesa: ORG, atletico mineiro: ORG",True
448,robustness,lowercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,cricket - lara endures another miserable day .,lara: PER,lara: PER,True
449,robustness,lowercase,Robert Galvin,robert galvin,robert galvin: PER,robert galvin: PER,True
450,robustness,lowercase,MELBOURNE 1996-12-06,melbourne 1996-12-06,melbourne: LOC,melbourne: LOC,True


In [30]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,62,164,73%,65%,True
1,robustness,lowercase,0,226,100%,65%,True



We can see that after performing augmentation, even the **add_typo** test is passing which failed earlier.